In [1]:
# !pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
# from tqdm.notebook import tqdm

if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2940, maxbars=100000, codepage=0, ping_last=161187, community_balance=0.0, retransmission=0.0, company='FX Choice Limited', name='FXChoice MetaTrader 5', language='English', path='C:\\Program Files\\FXChoice MetaTrader 5', ...)

In [2]:
risk = pd.read_csv('1y_risk.csv')
# risk

In [3]:
# get risk (size and SL dist)


In [4]:
# get risk * denomination * contract size

# denomination 
syms = set(list(risk['sym']) + ['GBPNZD'] + ['EURCAD'])
# syms

In [5]:
ticks = {}
for sym in syms:
    
    # print(sym)
    t = mt5.symbol_info_tick(sym)
    # print(t)
    p = (t.bid + t.ask) / 2
    ticks[sym] = p
    
#     if sym[3:] == 'USD':
#         print(sym, 1)
#     else:
#         print(sym, '1 / ' + sym[3:] + 'USD')

In [6]:
denom = {}
for sym in syms:
    if sym[3:] == 'USD':
        # print(sym, 1)
        denom[sym] = 1
    else:
        if sym[3:] + 'USD' in syms:
            # print(sym, sym[3:] + 'USD')
            denom[sym] = ticks[sym[3:] + 'USD']
        else:
            # print(sym, 'USD' + sym[3:], 1/ticks['USD' + sym[3:]])
            denom[sym] = 1/ticks['USD' + sym[3:]]

In [7]:
# denom

In [8]:
risk = risk.to_dict(orient='records')

In [9]:
# risk * denom * contract

scaled_mvmt = {}

for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        r = list(filter(lambda x: x['sym'] == sym, risk))[0]['avg+std']
        if sym[3:] == 'JPY':
            r = r/100
        if sym[3:] != 'JPY' and sym != 'BTCUSD' and sym != 'XAUUSD':
            r = r/10000
        c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 100000
        scaled_mvmt[sym] = r * denom[sym] * c

# scaled_mvmt

In [37]:
# account balance
account = mt5.account_info()

# account.balance

btc = mt5.symbol_info_tick('BTCUSD')
btc_mid = (btc.bid + btc.ask) / 2
btc_mid

usd = account.balance * btc_mid * 0.000001

fr = 0.025

dr = usd * fr

In [38]:
dr

10.634021844750002

In [12]:
size = {}
for sym in syms:
    if sym in [_['sym'] for _ in risk]:
        # print(sym, dr/scaled_mvmt[sym])
        size[sym] = round(dr/scaled_mvmt[sym], 2)

In [13]:
size

{'BTCUSD': 0.01,
 'GBPUSD': 0.03,
 'GBPCAD': 0.03,
 'EURCHF': 0.07,
 'EURGBP': 0.04,
 'AUDUSD': 0.05,
 'EURAUD': 0.04,
 'CADCHF': 0.07,
 'GBPCHF': 0.03,
 'EURNZD': 0.04,
 'EURUSD': 0.05,
 'CADJPY': 0.06,
 'NZDCHF': 0.06,
 'CHFJPY': 0.05,
 'NZDUSD': 0.05,
 'USDCHF': 0.06,
 'AUDJPY': 0.05,
 'GBPAUD': 0.03,
 'AUDCHF': 0.06,
 'NZDJPY': 0.06,
 'EURJPY': 0.05,
 'USDJPY': 0.07,
 'USDCAD': 0.05,
 'XAUUSD': 0.01,
 'AUDCAD': 0.07,
 'GBPJPY': 0.03}

In [14]:
dist = {}
for sym in syms:
    c = 100 if sym == 'XAUUSD' else 1 if sym == 'BTCUSD' else 1000 if sym[3:] == 'JPY' else 100000
    if sym in [_['sym'] for _ in risk]:
        # print(sym, scaled_mvmt[sym] / c)
        if sym[3:] == 'JPY':
            dist[sym] = round(scaled_mvmt[sym] / c, 3)
        else:
            dist[sym] = round(scaled_mvmt[sym] / c, 5)

In [15]:
dist

{'BTCUSD': 861.8,
 'GBPUSD': 0.0035,
 'GBPCAD': 0.0032,
 'EURCHF': 0.00159,
 'EURGBP': 0.00265,
 'AUDUSD': 0.00221,
 'EURAUD': 0.00266,
 'CADCHF': 0.00161,
 'GBPCHF': 0.00319,
 'EURNZD': 0.00262,
 'EURUSD': 0.00234,
 'CADJPY': 0.17,
 'NZDCHF': 0.00178,
 'CHFJPY': 0.197,
 'NZDUSD': 0.002,
 'USDCHF': 0.00192,
 'AUDJPY': 0.207,
 'GBPAUD': 0.00326,
 'AUDCHF': 0.00192,
 'NZDJPY': 0.186,
 'EURJPY': 0.22,
 'USDJPY': 0.162,
 'USDCAD': 0.00209,
 'XAUUSD': 9.49,
 'AUDCAD': 0.0016,
 'GBPJPY': 0.327}

In [16]:
risk_new = {}
for s in size.keys():
    risk_new[s] = {'size': size[s]}
for d in dist.keys():
    risk_new[d]['dist'] = dist[d]

In [39]:
risk_new['EURCAD'] = {'size': 0.03, 'dist': .0015}

In [34]:
def update_order(tradeorder):
    
    # get the new params
    print(tradeorder.symbol)
    
    # check if params different
    print(risk_new[tradeorder.symbol])
    
    # will fail for sells
    current_dist = round(abs(tradeorder.price_open - tradeorder.sl), 4)
    print('current dist:', current_dist)
    # if different, update to new
    
    # if no changes, return
    if round(risk_new[tradeorder.symbol]['dist'], 4) == current_dist and risk_new[tradeorder.symbol]['size'] == tradeorder.volume_current:
        return None
    
    # if duplicates return none
    
    
    # set new_vol to current in case no change
    new_vol = tradeorder.volume_current
    # set new_sl to current in case no change
    new_sl = tradeorder.sl
    
    if risk_new[tradeorder.symbol]['dist'] != current_dist:
        # print('  change dist')
        if tradeorder.type == 2:
            # print('    new buy sl')
            new_sl = tradeorder.price_open - risk_new[tradeorder.symbol]['dist']
            # print('    ', new_sl)
            
            o_type = mt5.ORDER_TYPE_BUY_LIMIT
            
        elif tradeorder.type == 3:
            # print('    new sell sl')
            new_sl = tradeorder.price_open + risk_new[tradeorder.symbol]['dist']
            # print('    ', new_sl)
            
            o_type = mt5.ORDER_TYPE_SELL_LIMIT
        
    if risk_new[tradeorder.symbol]['size'] != tradeorder.volume_current:
        # print('  change volume')
        new_vol = risk_new[tradeorder.symbol]['size']
        # print('    ', new_vol)
    
    # cancel order
    r = {
        "action": mt5.TRADE_ACTION_REMOVE,
        "order": tradeorder.ticket
    }
    result = mt5.order_send(r) 
    print('cancel:', result)
    
    r = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": tradeorder.symbol,
        "volume": new_vol,
        "type": o_type,
        "price": tradeorder.price_open,
        "sl": new_sl,
        # "tp": _[1] + (_[3]*3),
        "deviation": 10,
        "magic": 9,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(r) 
    print('cancel:', result)
    
    return result

In [35]:
# get current working orders
o = mt5.orders_get()

In [36]:
for _ in o:
    update_order(_)

CHFJPY
{'size': 0.05, 'dist': 0.197}
current dist: 0.197
USDJPY
{'size': 0.07, 'dist': 0.162}
current dist: 0.162
USDJPY
{'size': 0.07, 'dist': 0.162}
current dist: 0.162
XAUUSD
{'size': 0.01, 'dist': 9.49}
current dist: 9.49
USDJPY
{'size': 0.07, 'dist': 0.162}
current dist: 0.162
GBPUSD
{'size': 0.03, 'dist': 0.0035}
current dist: 0.0035
EURJPY
{'size': 0.05, 'dist': 0.22}
current dist: 0.22
EURCAD
{'size': 0.05, 'dist': 0.0015}
current dist: 0.0015
EURAUD
{'size': 0.04, 'dist': 0.00266}
current dist: 0.0027
CADCHF
{'size': 0.07, 'dist': 0.00161}
current dist: 0.0016
EURUSD
{'size': 0.05, 'dist': 0.00234}
current dist: 0.0023
GBPCHF
{'size': 0.03, 'dist': 0.00319}
current dist: 0.0032
GBPAUD
{'size': 0.03, 'dist': 0.00326}
current dist: 0.0033
EURNZD
{'size': 0.04, 'dist': 0.00262}
current dist: 0.0026
EURAUD
{'size': 0.04, 'dist': 0.00266}
current dist: 0.0027
USDCHF
{'size': 0.06, 'dist': 0.00192}
current dist: 0.0019
